In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('patent_abstract_5000.csv')
df

,Unnamed: 0,publication_number,application_number,text,filing_date,new_appl_nbr,appln_id,app_nbr,filing,tech_field,...,fwd_cits7_xy,breakthrough,breakthrough_xy,generality,originality,radicalness,renewal,quality_index_4,quality_index_6,quality_rank
0,1117907,EP-3412977-A1,EP-18173805-A,A radiant module (1) comprising: a first hollo...,20180523,EP20180173805,494709760,EP20180173805,2018,30.0,...,0,NaN,NaN,NaN,0.777778,0.916667,1.0,0.317460,0.293740,0
1,836155,EP-3062394-A1,EP-15200990-A,An array antenna device of this disclosure inc...,20151218,EP20150200990,447596715,EP20150200990,2015,3.0,...,0,NaN,NaN,NaN,0.777778,0.083333,3.0,0.216450,0.312182,1
2,36339,EP-2591987-A1,EP-10854435-A,A vehicle hood structure capable of improving ...,20100708,EP20100854435,340657535,EP20100854435,2010,32.0,...,0,NaN,NaN,NaN,0.408163,0.000000,8.0,0.140097,0.243247,0
3,536885,EP-2863948-A2,EP-13807400-A,This invention relates generally to the genera...,20130621,EP20130807400,413456628,EP20130807400,2013,16.0,...,2,NaN,NaN,0.707031,0.838169,0.518812,5.0,0.702178,0.560783,1
4,433243,EP-2854677-A1,EP-13742496-A,Methods and implants to treat anterior cruciat...,20130530,EP20130742496,409439382,EP20130742496,2013,13.0,...,0,NaN,NaN,NaN,0.480000,0.400000,6.0,0.233974,0.267708,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,26522,EP-2514281-A1,EP-10838186-A,A plasma gun system comprising: a plasma gun c...,20101213,EP20100838186,334899458,EP20100838186,2010,23.0,...,0,NaN,NaN,NaN,0.853203,0.406780,9.0,0.315754,0.293119,0
4996,357146,EP-2814870-A1,EP-12813624-A,An amorphous copolyester comprising the reacti...,20121211,EP20120813624,380483196,EP20120813624,2012,17.0,...,0,NaN,NaN,NaN,0.625000,0.375000,6.0,0.120370,0.202268,0
4997,810878,EP-2996404-A1,EP-15183744-A,A communication device for handling a selectio...,20150903,EP20150183744,444259224,EP20150183744,2015,4.0,...,0,NaN,NaN,NaN,0.625000,0.500000,3.0,0.127451,0.207662,0
4998,374199,EP-2745969-A1,EP-12198175-A,The toll has a tool shank that is provided wit...,20121219,EP20120198175,379202539,EP20120198175,2012,26.0,...,0,NaN,NaN,NaN,0.458977,0.206897,7.0,0.199134,0.322634,1


In [4]:
conditions = [

    (df['quality_index_4'] < 0.3),
    (df['quality_index_4'] >= 0.3)
]

choices = ['0', '1']

In [5]:

df['quality_rank'] = np.select(condlist=conditions, choicelist=choices)

In [6]:
df = df[['text','quality_rank']]

In [7]:
df['text'] = df['text'].apply(str)

/home/lin_menghsien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Create a model

In [8]:
import numpy as np
import pandas as pd
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

In [9]:
# more CNN library
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant


In [10]:
MAX_SEQUENCE_LENGTH = 500  # [Steven] I added for the CNN, to take only the 500 words.
MAX_SENT_LENGTH = 100
MAX_SENTS = 15
MAX_NB_WORDS = 20000  # [Steven] this should be number of unique word / vocabulary
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [11]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", str(string))    
    string = re.sub(r"\'", "", str(string))    
    string = re.sub(r"\"", "", str(string))    
    return string.strip().lower()

In [12]:
data_train = df.sample(frac=1)[:5000]

In [13]:
data_train.shape

(5000, 2)

In [14]:
import nltk
from nltk import tokenize

abstracts = [] # abstracts is list of list of list to hold each sentences of each abstract (the most complete data)
labels = [] # label is just a list holding our label which is quality_index
texts = []  # texts to hold each complete abstract as list of list (note: abstract not breaking up to sentence level)
for idx in range(data_train.text.shape[0]): # for each row
    #text = BeautifulSoup(data_train.text[idx])
    #print(clean_str(str(data_train.iloc[idx]['text'])))
    text = clean_str(str(data_train.iloc[idx]['text'])) # text is each complete abstract
    texts.append(text) # texts to hold each complete abstract as list of string (note: abstract not breaking up to sentence level)
    sentences = tokenize.sent_tokenize(text) # sentences is list of string holding each complete sentence of one abstract (but it's just an intermediate variable, not used directly in later code)
    abstracts.append(sentences) # abstracts is list of list of string to hold each sentences of each abstract (the most complete data) (this is what we use )
    labels.append(data_train.iloc[idx]['quality_rank']) # label is just a list holding our label which is quality_index

In [15]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) # intend to use next line .fit_on_texts to index each word within specific abstract at current iteration/loop, the more frequent word has lower index number, it is a dictionary format, it's like a unique vocabulary index
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
# data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

In [16]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 17733 unique tokens.


In [17]:
len(sequences)

5000

In [18]:
#sequences

In [19]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post') # The default is pre-padding, should I try post-padding ? => padding='post', truncating='post'. I think it has to do with whether the beginning words are more important, or later words more important. You may want to try both approaches and compare result?
data

array([[  174,     7,    41, ...,     0,     0,     0],
       [    1,    26,    40, ...,     0,     0,     0],
       [    8,    79,     9, ...,     0,     0,     0],
       ...,
       [    2, 10045,   639, ...,     0,     0,     0],
       [    1,    26,    40, ...,     0,     0,     0],
       [    8,   104,   233, ...,     0,     0,     0]], dtype=int32)

In [20]:
data.shape

(5000, 500)

In [21]:
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (5000, 500)
Shape of label tensor: (5000, 2)


In [22]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [23]:
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))


Number of positive and negative reviews in traing and validation set
[2632. 1368.]
[677. 323.]


In [24]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [25]:
y_val

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [33]:
import os
os.getcwd()

'/home/lin_menghsien/team_repo'

In [34]:
# import GloVe word embedding
GLOVE_DIR = "/home/lin_menghsien/w266/project/data/glove"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [35]:
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [36]:
len(word_index) + 1

17734

In [37]:
# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))  # EMBEDDING_DIM = 100 since we import 100d GloVe
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [38]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [40]:
import os, sys, re, json, time, datetime, shutil
from numpy.random import seed
seed(5)
import tensorflow
tensorflow.random.set_seed(5)

In [41]:
# Parameter setting
epochs = 15
embed_dim = EMBEDDING_DIM  # already specify with EMBEDDING_DIM variable above
dense_layer_dims = [100]
dropout_rate = 0.2
num_filters = [100, 100, 100]
kernel_sizes = [48, 50, 52]
num_classes = labels.shape[1] # 

# Model building
start_time = time.time()
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
h = embedding_layer(sequence_input) # remember embedding_layer assign with GloVe's weight

conv_layers_for_all_kernel_sizes = []
counter = 0
for kernel_size, filters in zip(kernel_sizes, num_filters): 
    conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(h)
    conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers_for_all_kernel_sizes.append(conv_layer)
    counter += 1
    #print(conv_layers_for_all_kernel_sizes)
if counter == 1: # need this since layers.concatenate throw error when there is only 1 item in conv_layers_for_all_kernel_sizes
    h = conv_layer
else:
    h = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1) # Concat the feature maps from each different size.

    h = keras.layers.Dropout(rate=dropout_rate)(h) # Dropout can help with overfitting (improve generalization) by randomly 0-ing different subsets of values in the vector

for dim in dense_layer_dims:
    h = keras.layers.Dense(dim, activation = 'relu')(h) # [Steven] I believe this add additional fully-connected hidden layer with the hope to increase model accuracy

prediction = keras.layers.Dense(num_classes, activation='softmax')(h)

model = keras.Model(inputs=sequence_input, outputs=prediction)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # From information theory notebooks.
              metrics=['accuracy'])        # What metric to output as we train.

# train
model.reset_states()
model.fit(x_train, 
          y_train, 
          epochs=epochs, 
          validation_data=(x_val, y_val))


Epoch 1/15
125/125 [==============================] - 257s 2s/step - loss: 0.8257 - accuracy: 0.6062 - val_loss: 0.6348 - val_accuracy: 0.6770
Epoch 2/15
125/125 [==============================] - 256s 2s/step - loss: 0.6109 - accuracy: 0.6662 - val_loss: 0.6257 - val_accuracy: 0.6730
Epoch 3/15
125/125 [==============================] - 257s 2s/step - loss: 0.5262 - accuracy: 0.7523 - val_loss: 0.6462 - val_accuracy: 0.6690
Epoch 4/15
125/125 [==============================] - 258s 2s/step - loss: 0.3415 - accuracy: 0.8708 - val_loss: 0.7040 - val_accuracy: 0.6000
Epoch 5/15
125/125 [==============================] - 256s 2s/step - loss: 0.1513 - accuracy: 0.9550 - val_loss: 0.8078 - val_accuracy: 0.6190
Epoch 6/15
125/125 [==============================] - 257s 2s/step - loss: 0.1182 - accuracy: 0.9615 - val_loss: 1.2512 - val_accuracy: 0.4410
Epoch 7/15
125/125 [==============================] - 257s 2s/step - loss: 0.0811 - accuracy: 0.9745 - val_loss: 0.9965 - val_accuracy: 0.6260